<h2> <b>Student ID: </b> 20009075
<h2> <b>Name: </b> Venkata Sandilya Bagavathula

**<h1><center> Crypto Coins Price Prediction using Machine Learning Model </center></h1>** 

<h3> <b>Import Required Libraries</b></h3>

In [1]:
import os
import pandas as pd
import numpy as np
import math
import datetime as dt
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM

import matplotlib.pyplot as plt
from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

<h3> <b>Loading Dataset </b></h3>

*   We will be using the dataset from Yahoo Finance to obtain data for crypto coins.
*   For our easy reference, here we're going with below Bitcoin prediction:
   https://finance.yahoo.com/quote/BTC-USD/history?p=BTC-USD
*   I've kept the dataset in the same directory.*italicized text*


In [3]:
maindf = pd.read_csv('/content/BTC-USD-2.csv')

<h3><b> Viewing the Dataset </b></h3>

In [4]:
print('Number of Days: ',maindf.shape[0])
print('Number of fields: ',maindf.shape[1])

Number of Days:  365
Number of fields:  7


In [5]:
# Starting data from the dataset
maindf.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-12-17,47653.730469,48004.894531,45618.214844,46202.144531,46202.144531,32902725329
1,2021-12-18,46219.253906,47313.828125,45598.441406,46848.777344,46848.777344,26098292690
2,2021-12-19,46853.867188,48089.664063,46502.953125,46707.015625,46707.015625,25154053861
3,2021-12-20,46707.062500,47401.718750,45579.808594,46880.277344,46880.277344,30961902129
4,2021-12-21,46886.078125,49300.917969,46698.773438,48936.613281,48936.613281,27055803928


In [6]:
# Ending data from the dataset
maindf.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
360,2022-12-12,17102.500000,17212.564453,16899.394531,17206.437500,17206.437500,19617581341
361,2022-12-13,17206.441406,17930.085938,17111.763672,17781.318359,17781.318359,26634741631
362,2022-12-14,17782.066406,18318.531250,17739.513672,17815.650391,17815.650391,25534481470
363,2022-12-15,17813.644531,17846.744141,17322.589844,17364.865234,17364.865234,20964448341
364,2022-12-16,17367.843750,17505.146484,16811.851563,16887.666016,16887.666016,22201923584


In [7]:
print('Shape of Dataset: {}' .format(maindf.shape))


print('Data Description: {}' .format(maindf.describe()))

Shape of Dataset: (365, 7)
Data Description:                Open          High           Low         Close     Adj Close  \
count    365.000000    365.000000    365.000000    365.000000    365.000000   
mean   29581.777922  30159.889838  28888.048443  29497.569403  29497.569403   
std    10687.227334  10903.240133  10452.939457  10664.971633  10664.971633   
min    15782.300781  16253.047852  15599.046875  15787.284180  15787.284180   
25%    19988.789063  20348.412109  19616.814453  19986.712891  19986.712891   
50%    24126.136719  24822.628906  23733.500000  23957.529297  23957.529297   
75%    40026.023438  40777.757813  39022.347656  39935.515625  39935.515625   
max    50854.917969  51956.328125  50514.496094  50822.195313  50822.195313   

             Volume  
count  3.650000e+02  
mean   3.053155e+10  
std    1.223760e+10  
min    1.270678e+10  
25%    2.335997e+10  
50%    2.857479e+10  
75%    3.448336e+10  
max    1.189925e+11  


<h3><b> Data Preprocessing </b></h3>

In [8]:
# Check for NA values
print('NA values:',maindf.isnull().values.any())

# Check for Null values
print('Null Values:',maindf.isnull().values.sum())

print('Shape of Dataset:', maindf.shape)

NA values: False
Null Values: 0
Shape of Dataset: (365, 7)


<h3><b>Model Building </b></h3>


*   Preparing data for training and testing.
*   We'll consider 1 year of data for training.
*   To predict the closing price, we'll consider only the close price and date.



<h3> <b>Displaying Bitcoin Price from 2014 - 2022 </b><h3>

In [9]:
# Getting all the closing prices from last one year
closedf = maindf[['Date','Close']]
print("Shape of close dataframe:", closedf.shape)

fig = px.line(closedf, x=closedf.Date, y=closedf.Close,labels={'date':'Date','close':'Close Stock'})
fig.update_traces(marker_line_width=2, opacity=0.8, marker_line_color='red')
fig.update_layout(title_text='Bitcoin close price from 2014-2022', plot_bgcolor='white', font_size=15, font_color='blue')
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Shape of close dataframe: (365, 2)


In [10]:
# Getting all the closing price of bitcoin from last one year

closedf = closedf[closedf['Date'] > '2021-07-29']
close_stock = closedf.copy()
print("Total data for prediction: ",closedf.shape[0])

Total data for prediction:  365


In [ ]:
closedf

,Date,Close
2508,2021-07-30,42235.546875
2509,2021-07-31,41626.195313
2510,2021-08-01,39974.894531
2511,2021-08-02,39201.945313
2512,2021-08-03,38152.980469
...,...,...
2868,2022-07-25,21361.701172
2869,2022-07-26,21239.753906
2870,2022-07-27,22930.548828
2871,2022-07-28,23843.886719


<h3> <b>Displaying Bitcoin Price of Last 1 Year <b></h3>

In [11]:
fig = px.line(closedf, x=closedf.Date, y=closedf.Close,labels={'date':'Date','close':'Close Stock'})
fig.update_traces(marker_line_width=2, opacity=0.8, marker_line_color='red')
fig.update_layout(title_text='Bitcoin close price from last 1 Year', plot_bgcolor='white', font_size=15, font_color='blue')
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

<h3> <b> Normalising Data </b></h3>

In [12]:
del closedf['Date']
scaler = MinMaxScaler(feature_range=(0,1))
closedf = scaler.fit_transform(np.array(closedf).reshape(-1,1))
print(closedf.shape)

(365, 1)


<h3> <b>Slicing Data into Training and Test Data </b></h3>


*   We keep the training Data as 60%
*   We keep the test Data as 40%



In [13]:
training_size = int(len(closedf)*0.60)
test_size = len(closedf) - training_size
train_data, test_data = closedf[0:training_size,:], closedf[training_size:len(closedf),:1]
print("train_data: ", train_data.shape)
print("test_data: ", test_data.shape)

train_data:  (219, 1)
test_data:  (146, 1)


<h3>Transform the closed price based on time series analysis forecasting <h3>

In [14]:
# Converting an array of values into a dataset matrix

def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [15]:
time_step = 15
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test", y_test.shape)

X_train:  (203, 15)
y_train:  (203,)
X_test:  (130, 15)
y_test (130,)


In [16]:
# reshape input to be of the format [samples, time steps, features] which is required for LSTM

X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)

X_train:  (203, 15, 1)
X_test:  (130, 15, 1)


<h3><b> Model Building </b></h3>

In [17]:
model=Sequential()

model.add(LSTM(10,input_shape=(None,1),activation="relu"))

model.add(Dense(1))

model.compile(loss="mean_squared_error",optimizer="adam")

In [18]:
history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=200,batch_size=32,verbose=1)

Epoch 1/200
7/7 [==============================] - 2s 58ms/step - loss: 0.5197 - val_loss: 0.0166
Epoch 2/200
7/7 [==============================] - 0s 12ms/step - loss: 0.4794 - val_loss: 0.0134
Epoch 3/200
7/7 [==============================] - 0s 17ms/step - loss: 0.4440 - val_loss: 0.0113
Epoch 4/200
7/7 [==============================] - 0s 15ms/step - loss: 0.4144 - val_loss: 0.0096
Epoch 5/200
7/7 [==============================] - 0s 17ms/step - loss: 0.3888 - val_loss: 0.0083
Epoch 6/200
7/7 [==============================] - 0s 15ms/step - loss: 0.3694 - val_loss: 0.0069
Epoch 7/200
7/7 [==============================] - 0s 17ms/step - loss: 0.3534 - val_loss: 0.0057
Epoch 8/200
7/7 [==============================] - 0s 17ms/step - loss: 0.3379 - val_loss: 0.0045
Epoch 9/200
7/7 [==============================] - 0s 17ms/step - loss: 0.3227 - val_loss: 0.0038
Epoch 10/200
7/7 [==============================] - 0s 13ms/step - loss: 0.3056 - val_loss: 0.0035
Epoch 11/200
7/7 [=

In [19]:
### Lets Do the prediction and check performance metrics

train_predict=model.predict(X_train)
test_predict=model.predict(X_test)
train_predict.shape, test_predict.shape

5/5 [==============================] - 0s 4ms/step


((203, 1), (130, 1))

<h3><b>Model Evaluation</b></h3>

In [20]:
# Transform back to original form

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 

In [21]:
# Evaluation metrices RMSE and MAE

print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Train data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

Train data RMSE:  1543.574744488152
Train data MSE:  2382622.991821664
Train data MAE:  1186.6158501447046
-------------------------------------------------
Test data RMSE:  790.992024912933
Test data MSE:  625668.3834758619
Test data MAE:  571.9705829432693


<h3>Variance Regression Score</h3>

In [22]:
print("Train data explained variance regression score:", explained_variance_score(original_ytrain, train_predict))
print("Test data explained variance regression score:", explained_variance_score(original_ytest, test_predict))

Train data explained variance regression score: 0.9690977948639783
Test data explained variance regression score: 0.8802000688890438


<h3>R square score for regression</h3>

In [23]:
print("Train data R2 score:", r2_score(original_ytrain, train_predict))
print("Test data R2 score:", r2_score(original_ytest, test_predict))

Train data R2 score: 0.9673815769570681
Test data R2 score: 0.8548658608388141


<h3><b>Comparision of original stock close price and predicted close price</b></h3>

In [24]:
# shift train predictions for plotting

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting

testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])


plotdf = pd.DataFrame({'date': close_stock['Date'],
                       'original_close': close_stock['Close'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Stock price','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Train predicted data:  (365, 1)
Test predicted data:  (365, 1)


<h3><b> Predicting next 30 days <b></h3>

In [25]:
x_input=test_data[len(test_data)-time_step:].reshape(1,-1)
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

from numpy import array

lst_output=[]
n_steps=time_step
i=0
pred_days = 30
while(i<pred_days):
    
    if(len(temp_input)>time_step):
        
        x_input=np.array(temp_input[1:])
        #print("{} day input {}".format(i,x_input))
        x_input = x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        
        yhat = model.predict(x_input, verbose=0)
        #print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)
       
        lst_output.extend(yhat.tolist())
        i=i+1
        
    else:
        
        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        temp_input.extend(yhat[0].tolist())
        
        lst_output.extend(yhat.tolist())
        i=i+1
               
print("Output of predicted next days: ", len(lst_output))

last_days=np.arange(1,time_step+1)
day_pred=np.arange(time_step+1,time_step+pred_days+1)
print(last_days)
print(day_pred)

temp_mat = np.empty((len(last_days)+pred_days+1,1))
temp_mat[:] = np.nan
temp_mat = temp_mat.reshape(1,-1).tolist()[0]

last_original_days_value = temp_mat
next_predicted_days_value = temp_mat

last_original_days_value[0:time_step+1] = scaler.inverse_transform(closedf[len(closedf)-time_step:]).reshape(1,-1).tolist()[0]
next_predicted_days_value[time_step+1:] = scaler.inverse_transform(np.array(lst_output).reshape(-1,1)).reshape(1,-1).tolist()[0]

new_pred_plot = pd.DataFrame({
    'last_original_days_value':last_original_days_value,
    'next_predicted_days_value':next_predicted_days_value
})

names = cycle(['Last 15 days close price','Predicted next 30 days close price'])

fig = px.line(new_pred_plot,x=new_pred_plot.index, y=[new_pred_plot['last_original_days_value'],
                                                      new_pred_plot['next_predicted_days_value']],
              labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Compare last 15 days vs next 30 days',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Close Price')

fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Output of predicted next days:  30
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
[16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39
 40 41 42 43 44 45]


<h3><b>Plotting entire Closing Stock Price with next 30 days period of prediction</b></h3>

In [26]:
lstmdf=closedf.tolist()
lstmdf.extend((np.array(lst_output).reshape(-1,1)).tolist())
lstmdf=scaler.inverse_transform(lstmdf).reshape(1,-1).tolist()[0]

names = cycle(['Close price'])

fig = px.line(lstmdf,labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Plotting whole closing stock price with prediction',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Stock')

fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()